In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
if os.name != 'posix':
    plt.rc("font", family="Malgun Gothic")
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import optuna
from sklearn.naive_bayes import GaussianNB

In [2]:
sk_label=pd.read_csv('C:/Users/fiona/Desktop/SK_label.csv', encoding='UTF-8')

In [3]:
sk=sk_label.drop(['year','month','day','day1_label','day3_label','대비','코스피대비'],axis=1)
sk.shape

(1429, 40)

In [4]:
label=pd.DataFrame()
label['encoding'] = sk_label['3일 등락률'].apply(lambda x: 1 if abs(x) < 5 else 0 if x >= 5 else 2)

In [5]:
sk1=sk.shift(1)
df=pd.concat([sk1,label],axis=1)
df=df.drop(0,axis=0)
df=df.reset_index(drop=True)

In [6]:
x=df.drop(['날짜','등락률','3일 등락률','encoding'],axis=1)
y=df.encoding

In [7]:
print('매수 encoding 갯수 : ' ,(label['encoding']==0).sum())
print('유지 encoding 갯수 : ' ,(label['encoding']==1).sum())
print('매도 encoding 갯수 : ' ,(label['encoding']==2).sum())

매수 encoding 갯수 :  155
유지 encoding 갯수 :  1167
매도 encoding 갯수 :  107


In [ ]:
#0.11
#0.82
#0.07

In [8]:
tscv=TimeSeriesSplit()
TimeSeriesSplit(n_splits=5)

for i, (train_index, test_index) in enumerate(tscv.split(x)):
    x_train, y_train=x.iloc[train_index],y.iloc[train_index]
    x_test, y_test=x.iloc[test_index],y.iloc[test_index]
    
    gnb=GaussianNB(priors=[11/100,82/100,7/100])
    fitted=gnb.fit(x_train,y_train)

    y_pred = fitted.predict(x_test)
    print(f"Fold {i+1} - Accuracy: {f1_score(y_test, y_pred,average='weighted')}")

Fold 1 - Accuracy: 0.7508207571156246
Fold 2 - Accuracy: 0.7528870922891057
Fold 3 - Accuracy: 0.6183696001977355
Fold 4 - Accuracy: 0.7513780999041498
Fold 5 - Accuracy: 0.7588275567334276


In [9]:
confusion_matrix(y_test,y_pred)

array([[  4,  23,   0],
       [  0, 189,   3],
       [  0,  18,   1]], dtype=int64)